In [1]:
!pip install cloud-tpu-client
!pip install dropconnect-tensorflow
!pip install tensorflow-addons


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.6 MB/s eta 0:00:00a 0:00:01
  Created wheel for dropconnect-tensorflow: filename=dropconnect_tensorflow-0.1.1-py3-none-any.whl size=4640 sha256=dcf3d4667712e9824cf3cf5451397d2bb3533598aef53841ca208a841013e047
  Stored in directory: /root/.cache/pip/wheels/7e/4a/e5/266cd645dbc3573352598ba045a07b58db04fc73b8cfab99ae
Successfully built dropconnect-tensorflow
  Attempting uninstall: keras
    Found existing installation: keras 3.0.5
    Uninstalling keras-3.0.5:
      Successfully uninstalled keras-3.0.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-nlp 0.8.1 requires keras-core, which is not installed.

[notice] A new release of pip is available: 2

In [2]:
import math
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import BatchNormalization
from keras.datasets import mnist
import tensorflow_addons as tfa
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Activation
from PIL import Image
import cv2
import numpy as np
from tensorflow.keras.layers import Dense, Input
from dropconnect_tensorflow import DropConnectDense
from tensorflow.keras.layers import (BatchNormalization, Conv2D, Dense, Dropout, Flatten, Input, Lambda, MaxPooling2D, Reshape)
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [7]:
warnings.filterwarnings('ignore')
AUTO = tf.data.experimental.AUTOTUNE

# Detect TPU, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


2024-03-03 13:08:39.808557: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-03-03 13:08:39.808667: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-03-03 13:08:39.808740: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-03-03 13:08:39.808815: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-03-03 13:08:39.808894: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-03-03 13:08:39.809106: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-03-03 13:08:39.809193: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this 

INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


REPLICAS:  8


In [8]:
def get_mat(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    # returns 3x3 transform matrix which transforms indicies
        
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    
    # ROTATION MATRIX
    c1 = tf.math.cos(rotation)
    s1 = tf.math.sin(rotation)
    one = tf.constant([1],dtype='float32')
    zero = tf.constant([1],dtype='float32')
    rotation_matrix = tf.reshape(tf.concat([c1,s1,zero, -s1,c1,zero, zero,zero,one],axis=0),[3,3])
    
    # SHEAR MATRIX
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)
    shear_matrix = tf.reshape(tf.concat([one,s2,zero, zero,c2,zero, zero,zero,one],axis=0),[3,3])    
    
    # ZOOM MATRIX
    zoom_matrix = tf.reshape(tf.concat([one/height_zoom,zero,zero, zero,one/width_zoom,zero, zero,zero,one],axis=0),[3,3])
    
    # SHIFT MATRIX
    shift_matrix = tf.reshape(tf.concat([one,zero,height_shift, zero,one,width_shift, zero,zero,one],axis=0),[3,3])

    return(rotation_matrix)

In [9]:
def transform(image,label):
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated
    DIM = image.shape[0]
    XDIM = DIM%2 #fix for size 331
    rot = 10. * tf.random.normal([1],dtype='float32')
    shr = 3. * tf.random.normal([1],dtype='float32') 
    h_zoom = .08 + tf.random.normal([1],dtype='float32')/10.
    w_zoom = .08 + tf.random.normal([1],dtype='float32')/10.
    random_factor = tf.random.uniform([1], minval=0, maxval=1, dtype='float32')

    # Interpolate within the range for height and width zoom factors
    h_shift = 8. * tf.random.normal([1],dtype='float32') 
    w_shift = 8. * tf.random.normal([1],dtype='float32') 
  
    # Get transformation matrix
    m = get_mat(rot, shr, h_zoom, w_zoom, h_shift, w_shift) 

    # List destination pixel indices
    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # Rotate destination pixels onto origin pixels
    idx2 = tf.keras.backend.dot(m,tf.cast(idx,dtype='float32'))
    idx2 = tf.keras.backend.cast(idx2,dtype='int32')
    idx2 = tf.keras.backend.clip(idx2,-DIM//2+XDIM+1,DIM//2)
    
    # Find origin pixel values           
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tf.gather_nd(image,tf.transpose(idx3))
        
    return tf.reshape(d,[DIM,DIM,1]),label



In [14]:
(x_train, y_train), (x_test, y_test) = mnist.load_data() 

# One-hot encode labels
num_classes = 10
batch_size = 32 * strategy.num_replicas_in_sync
epochs = 10

# Reshape the data to add the channel dimension
x_train_split = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
x_test_split = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0
y_train_split = to_categorical(y_train, num_classes)
y_test_split = to_categorical(y_test, num_classes)

In [11]:
# Put data in a tensor format for parallelization
train_dataset_augment = (
    tf.data.Dataset
    .from_tensor_slices((x_train_split.astype(np.float32), y_train_split.astype(np.float32)))
    .map(transform, num_parallel_calls=AUTO)
    .repeat()
    .shuffle(2048)
    .batch(batch_size)
    .prefetch(AUTO)
)

train_dataset = (
    tf.data.Dataset
    # The given tensors are sliced along their first dimension. This operation preserves the structure of the input tensors, 
    # removing the first dimension of each tensor and using it as the dataset dimension.
    .from_tensor_slices((x_train_split.astype(np.float32), 
                         y_train_split.astype(np.float32)))
    .repeat()
    .shuffle(2048)
    .batch(batch_size)
    .prefetch(AUTO)
)

val_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_test_split.astype(np.float32), 
                         y_test_split.astype(np.float32)))
    .batch(batch_size)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test_split.astype(np.float32))
    .batch(batch_size)
)



In [15]:

# Input layer
train_datasets_combined = tf.data.experimental.sample_from_datasets([train_dataset_augment, train_dataset], weights=[0.5, 0.5])

with strategy.scope():
    inputs = Input(shape=(28, 28, 1))
    
    # First Block
    x = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='valid')(inputs)
    x = BatchNormalization()(x)
    x = Dropout(0.35)(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.35)(x)
    x = Conv2D(256, kernel_size=(3, 3), activation='relu', padding='valid')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    # Second Block
    x = Dropout(0.35)(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='valid')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.35)(x)
    x = Conv2D(1024, (3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.35)(x)
    x = Conv2D(2000, (3, 3), activation='relu', padding='valid')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.35)(x)
    # Third Block - Main CNN is separated in order to keep with the main  path
    CNN = x
    CNN = Flatten()(CNN)
    CNN = Dense(512, activation='relu')(CNN)
    CNN = BatchNormalization()(CNN)
    CNN = Dropout(0.5)(CNN)
    CNN = DropConnectDense(512, activation='relu', prob=0.5)(CNN)
    CNN = Dense(10, activation='softmax')(CNN)
    '''
        At this point, the base CNN is done, but we need to go back to (x) which will be divided
            into 10 FCSNs
    '''
    # Prepare to divide the feature maps into 10 subnets
    shape = K.int_shape(x)
    num_feature_maps = shape[-1]  # This would be 2000 based on your architecture
    subnet_feature_maps = num_feature_maps // 10  # This assumes an even split

    # Create subnetworks
    subnet_outputs = []
    for i in range(10):
        # Extract the subset of feature maps for this subnet
        subnet_input = Lambda(lambda z: z[:, :, :, i * subnet_feature_maps:(i + 1) * subnet_feature_maps])(x)
        # Reshape the feature maps to a 1D vector
        subnet_input = Reshape((shape[1] * shape[2] * subnet_feature_maps,))(subnet_input)

        # Subnetwork layers
        fc = Dense(512, activation='relu')(subnet_input)
        fc = BatchNormalization()(fc)
        fc = Dropout(0.5)(fc)
        fc = DropConnectDense(512, activation='relu', prob=0.5)(fc)
        subnet_output = Dense(10, activation='softmax')(fc)
        subnet_outputs.append(subnet_output)
        
    subnet_outputs.append(CNN)    

    full_model = Model(inputs=inputs, outputs=subnet_outputs)
    full_model.summary()

    # Define AdamW optimizer from TensorFlow addons
    adamw_optimizer = tfa.optimizers.AdamW(
        learning_rate=0.001,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-8,
        weight_decay=0.0   # Set the weight decay rate here
    )

    # Compile the model

full_model.compile(loss='categorical_crossentropy', optimizer=adamw_optimizer, metrics=['accuracy'])



# Callbacks
def scheduler(epoch, lr):
    if epoch < 15: # For the first 10 epochs, the learning rate is not changed
        return(lr)
    elif 15 < epoch < 30: # After ten is decreases exponentially
        return(lr*math.exp(-0.1))
    else:
        return(lr*math.exp(-0.2)) # And then decreases even more, still exponentially

lr_scheduler = LearningRateScheduler(scheduler)
checkpoint = ModelCheckpoint('mnist_best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)



history = full_model.fit(train_datasets_combined, 
                    steps_per_epoch = len(x_train_split) / batch_size, 
                    epochs = epochs, 
                    validation_data=(val_dataset),
                    #callbacks = [checkpoint, lr_scheduler]
                    )
    

# Evaluate the model
#with strategy.scope():
#full_model.load_weights('mnist_best_model.h5')
    
score = full_model.evaluate(val_dataset)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
full_model.save("modelinho.h5")


def manual_majority_vote(predictions):
    # Assuming predictions shape is (num_subnets, num_samples, num_classes)
    # We first transpose to get shape (num_samples, num_subnets, num_classes)
    predictions = np.transpose(predictions, (1, 0, 2))
    
    # Now we get the class with the highest probability from each subnet
    votes = np.argmax(predictions, axis=-1)  # Shape becomes (num_samples, num_subnets)
    
    # Apply majority voting across subnets
    final_predictions = np.array([np.bincount(votes[i]).argmax() for i in range(votes.shape[0])])
    return final_predictions

# Load the model with custom DropConnectDense layer
model_for_inference = load_model("modelinho.h5", custom_objects={"DropConnectDense": DropConnectDense})

# Making predictions with the model
# Ensure x_test_split is preprocessed as the model expects
predictions = model_for_inference.predict(x_test_split)  # Output shape should be (num_samples, num_subnets, num_classes)

# Check if predictions are a list of arrays and stack them if necessary
if isinstance(predictions, list):
    predictions = np.stack(predictions, axis=0)

# Apply majority voting to these predictions
final_predictions = manual_majority_vote(predictions)

# Assuming y_test_split is one-hot encoded, convert it to class indices for comparison
y_test_indices = np.argmax(y_test_split, axis=1)

# Calculate accuracy
accuracy = np.mean(final_predictions == y_test_indices)
print(f"Test Accuracy after Majority Voting: {accuracy * 100:.2f}%")

CNN Shape: KerasTensor(type_spec=TensorSpec(shape=(None, 10), dtype=tf.float32, name=None), name='dense_23/Softmax:0', description="created by layer 'dense_23'")
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 28, 28, 1)]          0         []                            
                                                                                                  
 conv2d_6 (Conv2D)           (None, 26, 26, 64)           640       ['input_2[0][0]']             
                                                                                                  
 batch_normalization_17 (Ba  (None, 26, 26, 64)           256       ['conv2d_6[0][0]']            
 tchNormalization)                                                                                
                             

2024-03-03 13:14:44.244438: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AdamW/AdamW/AssignAddVariableOp.


234/234 [==============================] - 68s 101ms/step - loss: 2.8724 - dense_25_loss: 0.2628 - dense_27_loss: 0.2627 - dense_29_loss: 0.2627 - dense_31_loss: 0.2643 - dense_33_loss: 0.2649 - dense_35_loss: 0.2651 - dense_37_loss: 0.2649 - dense_39_loss: 0.2637 - dense_41_loss: 0.2676 - dense_43_loss: 0.2633 - dense_23_loss: 0.2305 - dense_25_accuracy: 0.9167 - dense_27_accuracy: 0.9154 - dense_29_accuracy: 0.9160 - dense_31_accuracy: 0.9159 - dense_33_accuracy: 0.9148 - dense_35_accuracy: 0.9154 - dense_37_accuracy: 0.9151 - dense_39_accuracy: 0.9161 - dense_41_accuracy: 0.9142 - dense_43_accuracy: 0.9151 - dense_23_accuracy: 0.9266 - val_loss: 28.3739 - val_dense_25_loss: 2.6699 - val_dense_27_loss: 3.0220 - val_dense_29_loss: 2.2070 - val_dense_31_loss: 2.7709 - val_dense_33_loss: 2.5552 - val_dense_35_loss: 2.4461 - val_dense_37_loss: 2.6342 - val_dense_39_loss: 2.8845 - val_dense_41_loss: 2.1946 - val_dense_43_loss: 2.6833 - val_dense_23_loss: 2.3062 - val_dense_25_accuracy: 0.

NameError: name 'load_model' is not defined